In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12232300683147843161
]


In [4]:
import numpy as np 
import os
import cv2
import h5py
import keras.backend as K
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint
from keras import backend as keras
from tensorflow.keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.optimizers import SGD 

import matplotlib.pyplot as plt




Found GPU at: /device:GPU:0


In [5]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
    return 2 * isct / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

def down(input_layer, filters, pool=True):
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(input_layer)
    residual = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    if pool:
        max_pool = MaxPool2D()(residual)
        return max_pool, residual
    else:
        return residual

def up(input_layer, residual, filters):
    filters=int(filters)
    upsample = UpSampling2D()(input_layer)
    upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
    concat = Concatenate(axis=3)([residual, upconv])
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(concat)
    conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    return conv2

def get_unet_model(filters=64):

    # Make a custom U-nets implementation.
    filters = 64
    input_layer = Input(shape = [256,256, 3])
    layers = [input_layer]
    residuals = []

    # Down 1, 128
    d1, res1 = down(input_layer, filters)
    residuals.append(res1)
    filters *= 2

    # Down 2, 64
    d2, res2 = down(d1, filters)
    residuals.append(res2)
    filters *= 2

    # Down 3, 32
    d3, res3 = down(d2, filters)
    residuals.append(res3)
    filters *= 2

    # Down 4, 16
    d4, res4 = down(d3, filters)
    residuals.append(res4)
    filters *= 2

    # Down 5, 8
    d5 = down(d4, filters, pool=False)

    # Up 1, 16
    up1 = up(d5, residual=residuals[-1], filters=filters/2)
    filters /= 2

    # Up 2,  32
    up2 = up(up1, residual=residuals[-2], filters=filters/2)
    filters /= 2

    # Up 3, 64
    up3 = up(up2, residual=residuals[-3], filters=filters/2)
    filters /= 2

    # Up 4, 128
    up4 = up(up3, residual=residuals[-4], filters=filters/2)
    out = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(up4)
     
    model = Model(input_layer, out)
    return model

In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)
file = h5py.File('drive/MyDrive/Dataset_train.h5', 'r')
imgs_train = file.get('images')
print(imgs_train)
imgs_mask_train = file.get('masks')
imgs_train = np.array(imgs_train)
imgs_mask_train = np.array(imgs_mask_train)

******************************
Loading and preprocessing train data...
******************************
<HDF5 dataset "images": shape (2099, 256, 256, 3), type "|u1">


In [9]:
imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std 

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255  # scale masks to [0, 1]

In [10]:
model =get_unet_model(filters=64)

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

In [11]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer = adam, loss =BinaryCrossentropy(), metrics=['accuracy',dice_coef])

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
history =  model.fit(imgs_train, imgs_mask_train, batch_size=32, epochs=30, verbose=2, shuffle=True,
          validation_split=0.2,
          callbacks=[model_checkpoint]
          )

Epoch 1/30
53/53 - 53s - loss: 0.0621 - accuracy: 0.9245 - dice_coef: 0.7680 - val_loss: 0.1916 - val_accuracy: 0.8910 - val_dice_coef: 0.6583
Epoch 2/30
53/53 - 53s - loss: 0.0631 - accuracy: 0.9243 - dice_coef: 0.7611 - val_loss: 0.1919 - val_accuracy: 0.8907 - val_dice_coef: 0.6640
Epoch 3/30
53/53 - 53s - loss: 0.0611 - accuracy: 0.9247 - dice_coef: 0.7698 - val_loss: 0.1916 - val_accuracy: 0.8907 - val_dice_coef: 0.6624
Epoch 4/30
53/53 - 53s - loss: 0.0617 - accuracy: 0.9247 - dice_coef: 0.7699 - val_loss: 0.1879 - val_accuracy: 0.8903 - val_dice_coef: 0.6556
Epoch 5/30
53/53 - 53s - loss: 0.0607 - accuracy: 0.9249 - dice_coef: 0.7726 - val_loss: 0.1992 - val_accuracy: 0.8911 - val_dice_coef: 0.6526
Epoch 6/30
53/53 - 53s - loss: 0.0598 - accuracy: 0.9251 - dice_coef: 0.7804 - val_loss: 0.1947 - val_accuracy: 0.8908 - val_dice_coef: 0.6643
Epoch 7/30
53/53 - 53s - loss: 0.0590 - accuracy: 0.9253 - dice_coef: 0.7845 - val_loss: 0.2017 - val_accuracy: 0.8909 - val_dice_coef: 0.6605

In [13]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Dataset_test.h5 to Dataset_test.h5
User uploaded file "Dataset_test.h5" with length 14812131 bytes


In [18]:
file = h5py.File('Dataset_test.h5', 'r')
imgs_test = file.get('images')
imgs_test= np.array(imgs_test)

imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

print('*'*30)
print('Loading saved weights...')
print('*'*30)
#model.load_weights('weights.h5')

print('*'*30)
print('Predicting masks on test data...')
print('*'*30)
imgs_mask_test = model.predict(imgs_test, verbose=1)

******************************
Loading saved weights...
******************************
******************************
Predicting masks on test data...
******************************
3/3 [==============================] - 66s 6s/step


In [19]:
print('*' * 30)
print('Saving predicted masks to files...')
print('*' * 30)
pred_dir = 'Preds3'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for i, image in enumerate(imgs_mask_test):
    image = (image * 255).astype(np.uint8)
    cv2.imwrite(os.path.join(pred_dir, str(i + 1) + '_pred.png'), image)

******************************
Saving predicted masks to files...
******************************
